In [45]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import wordnet
import spacy
from numpy import dot
from numpy.linalg import norm

In [2]:
def remove_stopwords(line, stopword_set):
    new_words = []
    
    for word in word_tokenize(line):
        if word.lower() in stopword_set:
            continue
        new_words.append(word.lower())
    
    return ' '.join(new_words)

def word_count(lst):
    d = {}
    for line in lst:
        for word in line.split():
            d[word] = d.get(word,0) + 1
    return d


# https://gaurav5430.medium.com/using-nltk-for-lemmatizing-sentences-c1bfff963258
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return lemmatized_sentence

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None
    



In [3]:
df = pd.read_excel('Behold+product+data+04262021.xlsx')

In [4]:
df.head()

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,product_active
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,NaN,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,Our signature khadi shirt\navailable in black ...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,True
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,NaN,2021-03-09 18:43:10.457 UTC,https://collina-strada-2.myshopify.com/product...,Mid-length dress with ruffles and adjustable s...,Mid-length dress with ruffles and adjustable s...,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,True
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,IBI Slip On Raw Red Knit Sneaker Women,NaN,2021-02-10 02:58:59.591 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,False
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,Unknown,IBI Slip On Black Knit Sneaker Women,NaN,2021-02-10 03:40:52.842 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,False
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,Unknown,CATIBA PRO Skate Black Suede and Canvas Contra...,NaN,2021-02-10 16:55:13.024 UTC,https://cariuma.myshopify.com/products/catiba-...,NaN,NaN,CATIBA PRO Skate Black Suede and Canvas Contra...,False


In [5]:
df.fillna(value='Unknown',inplace=True)

In [6]:
df.isnull().sum()

product_id             0
brand                  0
brand_category         0
name                   0
details                0
created_at             0
brand_canonical_url    0
description            0
brand_description      0
brand_name             0
product_active         0
dtype: int64

In [7]:
df.columns

Index(['product_id', 'brand', 'brand_category', 'name', 'details',
       'created_at', 'brand_canonical_url', 'description', 'brand_description',
       'brand_name', 'product_active'],
      dtype='object')

In [8]:
#features = ['brand_category', 'name', 'details', 'description', 
            #'brand_description', 'brand_name']
features = ['details', 'description']
label = ['brand']

In [9]:
other_features = ['product_active','created_at']

In [10]:
df['combined'] = df[features].apply(lambda col: ' '.join(col.astype(str)), axis=1)

In [11]:
X = df[['combined']]
y = df[label]

In [12]:
regex = re.compile(r'[^\w\s\d]')
X['cleaned'] = X['combined'].apply(lambda x: regex.sub(' ', x))


<ipython-input-12-1fcbfa2597f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['cleaned'] = X['combined'].apply(lambda x: regex.sub(' ', x))


In [13]:
stp = set(stopwords.words('english'))
added = ['unknown','½ï']

stp.update(added)

a = X['cleaned'].apply(lambda x: remove_stopwords(x, stp))


In [14]:
X['cleaned']

0        Unknown Our signature khadi shirt\navailable i...
1        Unknown Mid length dress with ruffles and adju...
2           Unknown IBI Slip On Raw Red Knit Sneaker Women
3             Unknown IBI Slip On Black Knit Sneaker Women
4                                          Unknown Unknown
                               ...                        
61350    Feminine flat mules \nSquare shape  V line on ...
61351    Feminine flat mules \nSquare shape  V line on ...
61352    Feminine flat mules \nSquare shape  V line on ...
61353    Feminine flat mules \nSquare shape  V line on ...
61354    Feminine flat mules \nSquare shape  V line on ...
Name: cleaned, Length: 61355, dtype: object

In [15]:
X['removed'] = a

<ipython-input-15-4d8ba31a4ff8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['removed'] = a


In [16]:
lemmatizer = WordNetLemmatizer()

In [17]:
a = X['removed'].apply(lambda x: lemmatize_sentence(x))
a

0        [signature, khadi, shirt, available, black, wh...
1        [mid, length, dress, ruffle, adjustable, strap...
2              [ibi, slip, raw, red, knit, sneaker, woman]
3                 [ibi, slip, black, knit, sneaker, woman]
4                                                       []
                               ...                        
61350    [feminine, flat, mule, square, shape, v, line,...
61351    [feminine, flat, mule, square, shape, v, line,...
61352    [feminine, flat, mule, square, shape, v, line,...
61353    [feminine, flat, mule, square, shape, v, line,...
61354    [feminine, flat, mule, square, shape, v, line,...
Name: removed, Length: 61355, dtype: object

In [18]:
b = a.apply(lambda x: ' '.join(x))
b

0        signature khadi shirt available black white ea...
1        mid length dress ruffle adjustable strap bias ...
2                      ibi slip raw red knit sneaker woman
3                        ibi slip black knit sneaker woman
4                                                         
                               ...                        
61350    feminine flat mule square shape v line front g...
61351    feminine flat mule square shape v line front g...
61352    feminine flat mule square shape v line front g...
61353    feminine flat mule square shape v line front g...
61354    feminine flat mule square shape v line front g...
Name: removed, Length: 61355, dtype: object

In [19]:
regex = re.compile(r'\d')
b = b.apply(lambda x: regex.sub(' ',x))

regex_let = re.compile(r'\b\w\b')
b = b.apply(lambda x: regex_let.sub('',x))

In [20]:
d = word_count(b)
d = dict(sorted(d.items(), key=lambda item: item[1], reverse = True))
d

{'size': 56028,
 'make': 32978,
 'fit': 30339,
 'model': 26984,
 'wear': 26254,
 'dry': 21929,
 'detail': 21785,
 'cotton': 21718,
 'style': 21275,
 'waist': 16484,
 'length': 15738,
 'feature': 15353,
 'wash': 14504,
 'clean': 13795,
 'hand': 13745,
 'leather': 13647,
 'sleeve': 13264,
 'fabric': 12700,
 'true': 12074,
 'front': 11298,
 'back': 11112,
 'small': 10969,
 'dress': 10812,
 'line': 10635,
 'pocket': 10461,
 'top': 10374,
 'button': 10297,
 'print': 9707,
 'measurement': 9477,
 'design': 8925,
 'soft': 8912,
 'hip': 8874,
 'care': 8511,
 'closure': 8430,
 'high': 8335,
 'hem': 8260,
 'bust': 8251,
 'cm': 8140,
 'body': 8115,
 'shoulder': 8053,
 'look': 7561,
 'height': 7431,
 'perfect': 7379,
 'long': 7351,
 'classic': 7338,
 'one': 7192,
 'import': 7169,
 'cold': 7138,
 'polyester': 6829,
 'piece': 6825,
 'strap': 6653,
 'neck': 6617,
 'tie': 6266,
 'silhouette': 6233,
 'black': 6087,
 'usa': 5994,
 'use': 5975,
 'color': 5949,
 'material': 5626,
 'skirt': 5613,
 'cut': 55

In [21]:
df['text_feature'] = b

In [22]:
#df['created_at']
#df['time'] = pd.to_datetime(df['created_at'])


In [23]:
#df['month'] = df['time'].dt.month
#df['year'] = df['time'].dt.year

In [24]:
df_cleaned = df[['product_id','text_feature']]

In [25]:
df_cleaned.head()

,product_id,text_feature
0,01EX0PN4J9WRNZH5F93YEX6QAF,signature khadi shirt available black white ea...
1,01F0C4SKZV6YXS3265JMC39NXW,mid length dress ruffle adjustable strap bias ...
2,01EY4Y1BW8VZW51BWG5VZY82XW,ibi slip raw red knit sneaker woman
3,01EY50E27A0P5V6KCW01XPDB43,ibi slip black knit sneaker woman
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,


In [26]:
nlp = spacy.load("en_core_web_md")

In [27]:
products = df['text_feature']
products = list(products.values)
products[:5]

['signature khadi shirt available black white easy wear beach city promise top go warm weather item perfect blazer hand loom woven stripe khadi cotton slightly sheer get soft every wash ship first week april color white black length    width      one size fit grid khadi cotton',
 'mid length dress ruffle adjustable strap bias cut side seam invisible zipper make new york model wear size small     rise sylk rise sylk organic cellulose fiber make natural waste rise bush stem',
 'ibi slip raw red knit sneaker woman',
 'ibi slip black knit sneaker woman',
 '']

In [28]:
for idx, product in enumerate(products):
    print(nlp(product))
    print(nlp(product).vector[:10]) 
    
    if idx == 5: # stop printing after first 5 or so, takes a long time!
        break

signature khadi shirt available black white easy wear beach city promise top go warm weather item perfect blazer hand loom woven stripe khadi cotton slightly sheer get soft every wash ship first week april color white black length    width      one size fit grid khadi cotton
[ 0.02473384  0.17092726 -0.04278119 -0.08279135  0.00974824 -0.11862255
 -0.01787883  0.01147373 -0.14507791  1.2740564 ]
mid length dress ruffle adjustable strap bias cut side seam invisible zipper make new york model wear size small     rise sylk rise sylk organic cellulose fiber make natural waste rise bush stem
[-0.05637427  0.21426725 -0.12740019  0.06513175  0.03297567  0.02609427
  0.0103011   0.08292246 -0.12685698  1.1181664 ]
ibi slip raw red knit sneaker woman
[-0.05495586 -0.05422028 -0.00206357  0.09203858  0.10340714  0.36522242
  0.01399185  0.06167328 -0.08643807  0.9407757 ]
ibi slip black knit sneaker woman
[ 0.02971482 -0.11026468 -0.05143516  0.05936867  0.09287167  0.16826384
 -0.03581167  0.0

In [29]:
#for idx, product in enumerate(products):
    #print(nlp(product))
    #print(nlp(product).vector[:10]) 

signature khadi shirt available black white easy wear beach city promise top go warm weather item perfect blazer hand loom woven stripe khadi cotton slightly sheer get soft every wash ship first week april color white black length    width      one size fit grid khadi cotton
[ 0.02473384  0.17092726 -0.04278119 -0.08279135  0.00974824 -0.11862255
 -0.01787883  0.01147373 -0.14507791  1.2740564 ]
mid length dress ruffle adjustable strap bias cut side seam invisible zipper make new york model wear size small     rise sylk rise sylk organic cellulose fiber make natural waste rise bush stem
[-0.05637427  0.21426725 -0.12740019  0.06513175  0.03297567  0.02609427
  0.0103011   0.08292246 -0.12685698  1.1181664 ]
ibi slip raw red knit sneaker woman
[-0.05495586 -0.05422028 -0.00206357  0.09203858  0.10340714  0.36522242
  0.01399185  0.06167328 -0.08643807  0.9407757 ]
ibi slip black knit sneaker woman
[ 0.02971482 -0.11026468 -0.05143516  0.05936867  0.09287167  0.16826384
 -0.03581167  0.0

[ 0.00960728  0.09893442 -0.14722951  0.05654574  0.06569476 -0.04505778
  0.01212947 -0.02107233 -0.0738535   0.98789483]
zaayra top chic crop blouse deep  neckline wide smock waist puffed sleeves feature elastic cuff tie detail cut watercolor floral print cotton blend top pair perfectly coordinate elia pant make usa dry clean    cotton    rayon model measurement height     waist    bust    hip    model wear size 
[-0.01161913  0.12266286 -0.18510482  0.03637334  0.08430804 -0.03607929
  0.04753878  0.02034764 -0.07873847  0.83580905]
annika mini silhouette feature long ruched flounce sleeve semi sheer top fold collar detail button front double chest pocket ruffle detail smock elastic waist lead line mini skirt tiered ruffle versatile silhouette cut gorgeous watercolor floral print chiffon make usa dry clean     poly chiffon model measurement height     waist    bust    hip    model wear size 
[ 0.01038641  0.11908059 -0.1934511   0.12335268  0.12832007 -0.0509247
  0.10116152  0.0351

[ 6.41842633e-02  1.32342175e-01 -1.99044660e-01  1.13228671e-01
  1.11438096e-01 -6.17652461e-02  1.34421036e-01 -2.38489592e-04
 -5.44738630e-03  8.59797180e-01]
gianna new mini silhouette handcraft floral chiffon transition seamlessly day even sophisticated style feature high smock neckline long blouson sleeve cuff elasticated waistline lead tiered ruffle trim mini skirt make usa dry clean     poly chiffon model measurement height     waist    bust    hip    model wear size 
[ 0.0192272   0.13181062 -0.17557505  0.08712779  0.08582786 -0.05599288
  0.153761    0.03866584 -0.01550029  0.8513586 ]
maya dress perfect effortless mini silhouette season beyond handcraft pink floral cotton blend feature split  neckline delicate ruffle neck detail signature petal tie three quarter sleeve finish wide smocked cuff skirt feature delicate ruffle trim make usa dry clean    rayon    cotton model measurement height     waist    bust    hip    model wear size 
[-0.00128209  0.15586783 -0.14076589  

item detail heel measure approximately   mm     inch color black suede effect outer    cotton    polyester    polyurethane sole    synthetic rubber    mineral compost    additive line    polyurethane    viscose make italy size fit care fit true size italian size clean either vegan leather suede upper gently wipe damp cloth careful rub much advise use cleaning product soap polish treatment may cause damage upper sustainability base cradle consumer life cycle assessment lca conduct sc global service behalf tina black suede effect follow impact environment work offset impact     global warm potential gwo      kg co  eq pair drive typical passenger vehicle    mile offset impact     invest green tree reforestation project project date project help reforest one million acres marginal farmland mississippi alluvial valley plant    million tree certified sc global service fresh water consumption fw        pair run typical dishwasher    time offset impact     investing water restoration certific

item detail heel measure approximately    mm   inch color patchwork python effect outer    polyester    polyamide sole    synthetic rubber    mineral compost    additive line    polyurethane    viscose lightly cushion insole make italy size fit care fit true size italian size clean either vegan leather suede upper gently wipe damp cloth careful rub much advise use cleaning product soap polish treatment may cause damage upper sustainability base cradle consumer life cycle assessment lca conduct sc global service behalf amelia patchwork python effect follow impact environment work offset impact     global warm potential gwp     kg co  eq pair drive typical passenger vehicle    mile offset impact     invest green tree reforestation project project date project help reforest one million acres marginal farmland mississippi alluvial valley plant    million tree certified sc global service fresh water consumption fw        pair run typical dishwasher    time offset impact     investing water 

...

    cotton button tie closure dry clean expose back crop blouse tie back style act      length        overlap size model wear size model measurement height      bust    waist    hip    fit true size     cotton length        overlap size button tie closure dry clean model wear size model measurement height      bust    waist    hip    fit true size style act     
[ 0.08199143  0.2529085  -0.1378698   0.07622086 -0.01468838 -0.11044774
  0.04159457  0.04828346 -0.01461583  0.9364174 ]
    polyester tie closure dry clean eyelet blouse shoulder style act      length        size model wear size model measurement height      bust    waist    hip    fit true size     polyester length        size tie closure dry clean eyelet blouse shoulder model wear size model measurement height      bust    waist    hip    fit true size style act     
[ 0.09510054  0.26646873 -0.14835806  0.09632382 -0.0039983  -0.07113541
  0.07866758  0.06966231  0.00395973  0.8690593 ]
composition cap sleeve key hole one

[ 0.07202838  0.1569159  -0.1622532  -0.00698678  0.06108953 -0.06285398
  0.056267    0.01785803 -0.0942434   0.78823936]
shoulder menswear stripe top exhibit play floral motif sleeves added texture    cotton    polyester dry clean make india size   bust    waist    length      sophia      wear size  
[ 0.00867463  0.21251285 -0.17334414  0.0149234   0.00103219 -0.03161459
  0.09423463  0.06867926 -0.08053447  0.8097583 ]
   rayon    linen length        size button closure dry clean style acdr      model wear size model measurement height      bust    waist    hip    fit true size    rayon    linen length        size button closure dry clean model wear size model measurement height      bust    waist    hip    fit true size style acdr     
[ 0.09351169  0.2889868  -0.1462225   0.02409747 -0.03909867 -0.10470291
  0.08648524  0.029303   -0.02172121  0.6887456 ]
    polyester body line zipper closure dry clean shoulder dress floral print style acdr      length        size model wear siz

design updated version classic boater klint refresh color braid straw classic strike material never go style material straw hand braid thin strand sew together form continuous piece material block hat specs light pink wheat braid straw brim     crown        rise leather band gold screw closure make usa size    cm medium
[-0.05223964  0.09831329  0.06981748 -0.00290796 -0.05017859 -0.0624636
  0.06207263  0.03323136 -0.09299956  1.0785022 ]
lunchpail bucket bag classic cesta one start cesta basket handwoven female artisan rwanda africa use locally source renewable resource piece hand finish italy carefully choose sustainable material basket take master craftswoman   day weave            leather camel italian nappa vegetable tan basket organic vegetable dye sisal natural navy line  oz organic cotton canvas natural hardware handtooled   kt gold plat brass
[-0.16211884  0.03547163  0.0769405  -0.07536164  0.05843436  0.09158409
 -0.0359822  -0.00795813 -0.1660959   0.9685055 ]
lunchpail bu

[ 0.08167167  0.29099873 -0.131465    0.0160693  -0.04871335 -0.10910158
  0.09673788  0.05125533 -0.01786141  0.7486373 ]
    polyester length        size tie zipper closure body line dry clean style acdr      model wear size model measurement height      bust    waist    hip    fit true size     polyester length        size tie zipper closure body line dry clean model wear size model measurement height      bust    waist    hip    fit true size style acdr     
[ 0.11729814  0.28004292 -0.12987727  0.07518674  0.03792883 -0.06793499
  0.09269406  0.06983291  0.02226989  0.81378645]
    cotton length        size zipper closure body line dry clean one sleeve dress style acdr      model wear size model measurement height      bust    waist    hip    fit true size     cotton length        size zipper closure body line dry clean one sleeve dress model wear size model measurement height      bust    waist    hip    fit true size style acdr     
[ 0.12318456  0.2654011  -0.15443586  0.049448

KeyboardInterrupt: 

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(products)

tf_idf_lookup_table = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [31]:
DOCUMENT_SUM_COLUMN = "DOCUMENT_TF_IDF_SUM"

# sum the tf idf scores for each document
tf_idf_lookup_table[DOCUMENT_SUM_COLUMN] = tf_idf_lookup_table.sum(axis=1)
available_tf_idf_scores = tf_idf_lookup_table.columns # a list of all the columns we have
available_tf_idf_scores = set(map( lambda x: x.lower(), available_tf_idf_scores)) # lowercase everything

In [ ]:
tf_idf_lookup_table[:20]

In [33]:
len(products)

61355

In [36]:
products_vectors = []
i=0
for idx, product in enumerate(products): # iterate through each review
    tokens = nlp(product) # have spacy tokenize the review text
    
    # initially start a running total of tf-idf scores for a document
    total_tf_idf_score_per_document = 0
    
    # start a running total of initially all zeroes (300 is picked since that is the word embedding size used by word2vec)
    running_total_word_embedding = np.zeros(300) 
    for token in tokens: # iterate through each token
    
    # if the token has a pretrained word embedding it also has a tf-idf score
        if token.has_vector and token.text.lower() in available_tf_idf_scores:
            
            tf_idf_score = tf_idf_lookup_table.loc[idx, token.text.lower()]
            #print(f"{token} has tf-idf score of {tf_idf_lookup_table.loc[idx, token.text.lower()]}")
            running_total_word_embedding += tf_idf_score * token.vector
            
            total_tf_idf_score_per_document += tf_idf_score
    
    # divide the total embedding by the total tf-idf score for each document
    # print(total_tf_idf_score_per_document)
    print(i)
    document_embedding = running_total_word_embedding / max(1,total_tf_idf_score_per_document)
    products_vectors.append(document_embedding)
    i+=1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070


3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715


5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361


6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012


...

54745
54746
54747
54748
54749
54750
54751
54752
54753
54754
54755
54756
54757
54758
54759
54760
54761
54762
54763
54764
54765
54766
54767
54768
54769
54770
54771
54772
54773
54774
54775
54776
54777
54778
54779
54780
54781
54782
54783
54784
54785
54786
54787
54788
54789
54790
54791
54792
54793
54794
54795
54796
54797
54798
54799
54800
54801
54802
54803
54804
54805
54806
54807
54808
54809
54810
54811
54812
54813
54814
54815
54816
54817
54818
54819
54820
54821
54822
54823
54824
54825
54826
54827
54828
54829
54830
54831
54832
54833
54834
54835
54836
54837
54838
54839
54840
54841
54842
54843
54844
54845
54846
54847
54848
54849
54850
54851
54852
54853
54854
54855
54856
54857
54858
54859
54860
54861
54862
54863
54864
54865
54866
54867
54868
54869
54870
54871
54872
54873
54874
54875
54876
54877
54878
54879
54880
54881
54882
54883
54884
54885
54886
54887
54888
54889
54890
54891
54892
54893
54894
54895
54896
54897
54898
54899
54900
54901
54902
54903
54904
54905
54906
54907
54908
54909
54910
5491

56124
56125
56126
56127
56128
56129
56130
56131
56132
56133
56134
56135
56136
56137
56138
56139
56140
56141
56142
56143
56144
56145
56146
56147
56148
56149
56150
56151
56152
56153
56154
56155
56156
56157
56158
56159
56160
56161
56162
56163
56164
56165
56166
56167
56168
56169
56170
56171
56172
56173
56174
56175
56176
56177
56178
56179
56180
56181
56182
56183
56184
56185
56186
56187
56188
56189
56190
56191
56192
56193
56194
56195
56196
56197
56198
56199
56200
56201
56202
56203
56204
56205
56206
56207
56208
56209
56210
56211
56212
56213
56214
56215
56216
56217
56218
56219
56220
56221
56222
56223
56224
56225
56226
56227
56228
56229
56230
56231
56232
56233
56234
56235
56236
56237
56238
56239
56240
56241
56242
56243
56244
56245
56246
56247
56248
56249
56250
56251
56252
56253
56254
56255
56256
56257
56258
56259
56260
56261
56262
56263
56264
56265
56266
56267
56268
56269
56270
56271
56272
56273
56274
56275
56276
56277
56278
56279
56280
56281
56282
56283
56284
56285
56286
56287
56288
56289
5629

57491
57492
57493
57494
57495
57496
57497
57498
57499
57500
57501
57502
57503
57504
57505
57506
57507
57508
57509
57510
57511
57512
57513
57514
57515
57516
57517
57518
57519
57520
57521
57522
57523
57524
57525
57526
57527
57528
57529
57530
57531
57532
57533
57534
57535
57536
57537
57538
57539
57540
57541
57542
57543
57544
57545
57546
57547
57548
57549
57550
57551
57552
57553
57554
57555
57556
57557
57558
57559
57560
57561
57562
57563
57564
57565
57566
57567
57568
57569
57570
57571
57572
57573
57574
57575
57576
57577
57578
57579
57580
57581
57582
57583
57584
57585
57586
57587
57588
57589
57590
57591
57592
57593
57594
57595
57596
57597
57598
57599
57600
57601
57602
57603
57604
57605
57606
57607
57608
57609
57610
57611
57612
57613
57614
57615
57616
57617
57618
57619
57620
57621
57622
57623
57624
57625
57626
57627
57628
57629
57630
57631
57632
57633
57634
57635
57636
57637
57638
57639
57640
57641
57642
57643
57644
57645
57646
57647
57648
57649
57650
57651
57652
57653
57654
57655
57656
5765

58859
58860
58861
58862
58863
58864
58865
58866
58867
58868
58869
58870
58871
58872
58873
58874
58875
58876
58877
58878
58879
58880
58881
58882
58883
58884
58885
58886
58887
58888
58889
58890
58891
58892
58893
58894
58895
58896
58897
58898
58899
58900
58901
58902
58903
58904
58905
58906
58907
58908
58909
58910
58911
58912
58913
58914
58915
58916
58917
58918
58919
58920
58921
58922
58923
58924
58925
58926
58927
58928
58929
58930
58931
58932
58933
58934
58935
58936
58937
58938
58939
58940
58941
58942
58943
58944
58945
58946
58947
58948
58949
58950
58951
58952
58953
58954
58955
58956
58957
58958
58959
58960
58961
58962
58963
58964
58965
58966
58967
58968
58969
58970
58971
58972
58973
58974
58975
58976
58977
58978
58979
58980
58981
58982
58983
58984
58985
58986
58987
58988
58989
58990
58991
58992
58993
58994
58995
58996
58997
58998
58999
59000
59001
59002
59003
59004
59005
59006
59007
59008
59009
59010
59011
59012
59013
59014
59015
59016
59017
59018
59019
59020
59021
59022
59023
59024
5902

60228
60229
60230
60231
60232
60233
60234
60235
60236
60237
60238
60239
60240
60241
60242
60243
60244
60245
60246
60247
60248
60249
60250
60251
60252
60253
60254
60255
60256
60257
60258
60259
60260
60261
60262
60263
60264
60265
60266
60267
60268
60269
60270
60271
60272
60273
60274
60275
60276
60277
60278
60279
60280
60281
60282
60283
60284
60285
60286
60287
60288
60289
60290
60291
60292
60293
60294
60295
60296
60297
60298
60299
60300
60301
60302
60303
60304
60305
60306
60307
60308
60309
60310
60311
60312
60313
60314
60315
60316
60317
60318
60319
60320
60321
60322
60323
60324
60325
60326
60327
60328
60329
60330
60331
60332
60333
60334
60335
60336
60337
60338
60339
60340
60341
60342
60343
60344
60345
60346
60347
60348
60349
60350
60351
60352
60353
60354
60355
60356
60357
60358
60359
60360
60361
60362
60363
60364
60365
60366
60367
60368
60369
60370
60371
60372
60373
60374
60375
60376
60377
60378
60379
60380
60381
60382
60383
60384
60385
60386
60387
60388
60389
60390
60391
60392
60393
6039

In [ ]:
products_vectors[:20]

In [38]:
len(products_vectors[2])

300

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [39]:
query = 'slimming fit straight leg pants'

In [40]:
tokens = nlp(query) # have spacy tokenize the review text
# initially start a running total of tf-idf scores for a document
total_tf_idf_score_per_document = 0
# start a running total of initially all zeroes (300 is picked since that is the word embedding size used by word2vec)
running_total_word_embedding = np.zeros(300) 
for token in tokens: # iterate through each token
    if token.has_vector and token.text.lower() in available_tf_idf_scores:
        tf_idf_score = tf_idf_lookup_table.loc[idx, token.text.lower()]
        #print(f"{token} has tf-idf score of {tf_idf_lookup_table.loc[idx, token.text.lower()]}")
        running_total_word_embedding += tf_idf_score * token.vector
        total_tf_idf_score_per_document += tf_idf_score
query_embedding = running_total_word_embedding / max(1,total_tf_idf_score_per_document)

In [ ]:
running_total_word_embedding.sum()

In [43]:
total_tf_idf_score_per_document

0.0

In [ ]:
max_sim = 0
for product in products_vectors:
    cos_sim = dot(product, query_embedding)/(norm(product)*norm(query_embedding))
    if cos_sim > max_sim:
        max_sim = cos_sim
        product_select = product
